In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T
from time import perf_counter
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from tqdm import tqdm

In [2]:
def train(model: nn.Module, loader: DataLoader) -> float:
    model.train()

    train_loss = 0

    for x, y in tqdm(loader, desc='Train'):
        optimizer.zero_grad()

        output = model(x)

        loss = loss_fn(output, y)

        train_loss += loss.item()

        loss.backward()

        optimizer.step()

    train_loss /= len(loader)
    
    return train_loss

In [3]:
@torch.inference_mode()
def evaluate(model: nn.Module, loader: DataLoader) -> tuple[float, float]:
    model.eval()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(loader, desc='Evaluation'):
        output = model(x)

        loss = loss_fn(output, y)

        total_loss += loss.item()

        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    total_loss /= len(loader)
    accuracy = correct / total

    return total_loss, accuracy

In [4]:
def create_mlp_model():
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(28 * 28, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )

In [5]:
mnist_train = MNIST(
    "../datasets/mnist",
    train=True,
    download=True,
    transform=T.ToTensor()
)

In [6]:
mnist_valid = MNIST(
    "../datasets/mnist",
    train=False,
    download=True,
    transform=T.ToTensor()
)

In [7]:
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
valid_loader = DataLoader(mnist_valid, batch_size=64, shuffle=False)

In [12]:
first_model = create_mlp_model()

optimizer = torch.optim.Adam(first_model.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

In [22]:
num_epochs = 15

start = perf_counter()

for epoch in range(num_epochs):
    train_loss = train(first_model, train_loader)

    valid_loss, valid_accuracy = evaluate(first_model, valid_loader)
    
    print(train_loss, valid_loss, valid_accuracy)
    
    
print(f'Total training and evaluation time {perf_counter() - start:.5f}')

Train:  46%|█████████████████████████████████▌                                       | 432/938 [00:04<00:05, 99.56it/s]


KeyboardInterrupt: 

In [14]:
torch.save(first_model.state_dict(), 'model_weights.pth')

In [8]:
def create_conv_model():
    return nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Flatten(),
        nn.Linear(4 * 4 * 64, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )

In [9]:
second_model = create_conv_model()

optimizer = torch.optim.Adam(second_model.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

In [12]:
num_epochs = 15

train_loss_history, valid_loss_history = [], []
valid_accuracy_history = []

start = perf_counter()

while True:
    train_loss = train(second_model, train_loader)

    valid_loss, valid_accuracy = evaluate(second_model, valid_loader)
    
    print(train_loss, valid_loss, valid_accuracy)
    
    if valid_accuracy >= 0.993:
        break

print(f'Total training and evaluation time {perf_counter() - start:.5f}')

Evaluation: 100%|████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 54.81it/s]


0.0040521712821358 0.03216483933761354 0.9927


Evaluation: 100%|████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 53.58it/s]


0.003681065294181807 0.03970196750379224 0.9923


Evaluation: 100%|████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 53.42it/s]


...

0.003605999756824307 0.03549952484023789 0.9928


Evaluation: 100%|████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 52.63it/s]


0.004262440678779814 0.04807005376739393 0.9912


Evaluation: 100%|████████████████████████████████████████████████████████████████████| 157/157 [00:02<00:00, 53.37it/s]

0.0037775195057772764 0.036125421941666944 0.993
Total training and evaluation time 280.04801


In [13]:
torch.save(second_model.state_dict(), 'model_2_weights.pth')